In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from prophet import Prophet

In [2]:
df = pd.read_csv("Fare_Card_History_for_Metropolitan_Transportation_Authority__MTA___Beginning_2010.csv")
df.head()

,From Date,To Date,Remote Station ID,Station,Full Fare,Senior Citizen / Disabled,7 Day ADA Farecard Access System Unlimited,30 Day ADA Farecard Access System Unlimited,Joint Rail Road Ticket,7 Day Unlimited,...,LIB Special Senior,Rail Road Unlimited No Trade,Transit Check Metrocard Annual Metrocard,Mail and Ride Easy Pay Express,Mail and Ride Unlimited,Path 2 Trip,Airtran Full Fare,Airtran 30 Day,Airtran 10 Trip,Airtran Monthly
0,08/06/2021,07/31/2021,R001,WHITEHALL STREET,27946,1824,242,376,8,11958,...,539,162,0,717,0,0,0,327,0,0
1,08/06/2021,07/31/2021,R003,CYPRESS HILLS,1323,134,3,13,0,817,...,12,19,0,11,0,0,0,70,0,0
2,08/06/2021,07/31/2021,R004,75TH STREET & ELDERTS LANE,4241,321,15,43,0,2029,...,14,31,0,28,0,0,0,301,0,0
3,08/06/2021,07/31/2021,R005,85TH STREET & FOREST PKWAY,4418,389,29,30,1,1999,...,13,25,0,40,0,0,0,111,0,0
4,08/06/2021,07/31/2021,R006,WOODHAVEN BOULEVARD,4193,315,27,55,0,2946,...,36,14,0,25,0,0,0,148,0,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250874 entries, 0 to 250873
Data columns (total 26 columns):
 #   Column                                       Non-Null Count   Dtype  
---  ------                                       --------------   -----  
 0   From Date                                    250874 non-null  object 
 1   To Date                                      250874 non-null  object 
 2   Remote Station ID                            250874 non-null  object 
 3   Station                                      250874 non-null  object 
 4   Full Fare                                    250874 non-null  int64  
 5   Senior Citizen / Disabled                    250874 non-null  int64  
 6   7 Day ADA Farecard Access System Unlimited   250874 non-null  int64  
 7   30 Day ADA Farecard Access System Unlimited  250874 non-null  int64  
 8   Joint Rail Road Ticket                       250874 non-null  int64  
 9   7 Day Unlimited                              250874 non-nul

In [4]:
df['From Date'] = pd.to_datetime(df['From Date'], format='%m/%d/%Y')
df['To Date'] = pd.to_datetime(df['To Date'], format='%m/%d/%Y')
df.sort_values(by=["To Date"], inplace = True)
df

,From Date,To Date,Remote Station ID,Station,Full Fare,Senior Citizen / Disabled,7 Day ADA Farecard Access System Unlimited,30 Day ADA Farecard Access System Unlimited,Joint Rail Road Ticket,7 Day Unlimited,...,LIB Special Senior,Rail Road Unlimited No Trade,Transit Check Metrocard Annual Metrocard,Mail and Ride Easy Pay Express,Mail and Ride Unlimited,Path 2 Trip,Airtran Full Fare,Airtran 30 Day,Airtran 10 Trip,Airtran Monthly
250873,2010-05-29,2010-06-04,R552,PA-PATH JOURNAL SQUARE,50892,0,0,0,3,0,...,0,0,0,0,0,0,144,0,0,0
250556,2010-05-29,2010-06-04,R155,KINGSBRIDGE RD-196TH ST,15268,503,111,231,6,8071,...,0,15,1243,18,7,0,29,0,0,0
250557,2010-05-29,2010-06-04,R156,BEDFORD PARK BLVD-203RD ST,11519,533,103,227,1,5400,...,0,4,1278,16,6,0,24,0,0,0
250558,2010-05-29,2010-06-04,R157,205TH ST-BAINBRIDGE AVE,15870,529,114,291,2,7867,...,0,17,1375,18,31,0,95,0,0,0
250559,2010-05-29,2010-06-04,R158,UNION TURNPIKE-KEW GARDENS,43481,2523,281,1295,46,20319,...,1,157,4723,156,110,0,1356,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,2021-08-06,2021-07-31,R329,MORRIS PARK-PAULDING AVENUE,3162,199,9,32,0,806,...,33,46,0,14,0,0,0,68,0,0
319,2021-08-06,2021-07-31,R330,3RD AVENUE-14TH STREET,6467,395,36,95,2,2619,...,153,65,0,106,0,0,0,35,0,2
320,2021-08-06,2021-07-31,R331,155TH STREET-ST NICHOLAS AV,2537,307,26,67,0,1605,...,32,31,0,47,0,0,0,86,0,0
308,2021-08-06,2021-07-31,R319,63RD STREET-LEXINGTON AVE,18205,1993,124,234,1,6645,...,525,212,0,206,0,0,0,74,0,0


In [5]:
station_names = df["Station"].unique()
station_names

array(['PA-PATH JOURNAL SQUARE', 'KINGSBRIDGE RD-196TH ST',
       'BEDFORD PARK BLVD-203RD ST', '205TH ST-BAINBRIDGE AVE',
       'UNION TURNPIKE-KEW GARDENS', '116TH STREET-COLUMBIA',
       'ASTOR PLACE-4 AVENUE', 'KINGSBRIDGE RD-JEROME AVE',
       'ELDER AVE-WESTCHESTER AVE', '14TH STREET-6TH AVENUE',
       '66TH STREET-BROADWAY', 'TOMPKINSVILLE', '79TH STREET-BROADWAY',
       '86TH STREET-BROADWAY', '96TH STREET-BROADWAY',
       '137TH STREET-BROADWAY', '14TH STREET-UNION SQUARE',
       '7TH AVENUE-PARK PLACE', 'BRIGHTON BEACH-CONEY ISLAND',
       '207TH STREET-BROADWAY', '181ST STREET-FT WASH AVE',
       '8TH AVENUE-14TH STREET', '33RD STREET-PARK AVENUE',
       '68TH STREET-HUNTER COLLEGE', '77TH STREET-LEXINGTON AVE',
       '86TH STREET-LEXINGTON AVE', 'TREMONT AVE-179TH ST',
       '103RD STREET-LEXINGTON AVE', 'UTICA AVE-STUYVESANT AVE',
       'STILLWELL AVE-CONEY ISLAND', '175TH STREET/FT WASHINGTON',
       'JAY ST-METROTECH', 'SUTPHIN BLVD-HILLSIDE AVE',
       '

In [6]:
remote_station_ids = df["Remote Station ID"].unique()
remote_station_ids.sort()

In [7]:
cards = [

    "Senior Citizen / Disabled",
    "1 Day Unlimited",
    "7 Day Unlimited",
    "14 Day Unlimited",
    "30 Day Unlimited"
]

# You can access the items in the 'cards' list like this:
# For example, to print the first card:
# print(cards[0])


In [ ]:
for remote_station_id in remote_station_ids:
    data = df[df["Remote Station ID"]==remote_station_id]

    data.plot(x='To Date', y=cards, figsize = (20, 20))
    station = data.loc[data['Remote Station ID'] == remote_station_id, 'Station'].values[0]

    print()
    # Set the x-axis label
    plt.xlabel('Date')

    # Set the y-axis label
    plt.ylabel('Count')

    # Set the title of the plot
    plt.title(f'{station} (ID: {remote_station_id})')

    plt.savefig(f"plots/{remote_station_id}")
    # Display the plot
    plt.show()


In [15]:
# important_station_ids = ["R012","R027", "R032", "R034", "R044", "R087", "R159"]
important_station_ids = ["R012"]

In [16]:
prediction_headings = ["Date"]
for card in cards:
    prediction_headings.append(card)

In [26]:
for important_station_id in important_station_ids:
    data = df[df["Remote Station ID"]==important_station_id]
    # print(data[["To Date", card]])
    station = data.loc[data['Remote Station ID'] == important_station_id, 'Station'].values[0]
    predictions = pd.DataFrame(columns = prediction_headings)
    for card in cards:
        model = Prophet().add_country_holidays(country_name='US')
        train_data = data[["To Date", card]]
        train_data = train_data.rename(columns={"To Date":"ds",card:"y"})
        print(train_data)
        model.fit(train_data)

        future = model.make_future_dataframe(periods = 365)
        forecast = model.predict(future)
        predictions["Date"] = forecast["ds"]
        predictions[card] = forecast["yhat"]


    predictions.plot(x='Date', y=cards, figsize = (20, 20))
    


    # Set the x-axis label
    plt.xlabel('Date')

    # Set the y-axis label
    plt.ylabel('Count')

    # Set the title of the plot
    plt.title(f'{station} (ID: {important_station_id})')

    plt.savefig(f"predictions/{important_station_id}")
    # Display the plot
    plt.show()

23:08:08 - cmdstanpy - INFO - Chain [1] start processing


               ds     y
250422 2010-06-04  6239
249960 2010-06-11  6844
249496 2010-06-18  6895
249032 2010-06-25  6192
248567 2010-07-02  6357
...           ...   ...
1885   2020-11-27  2095
1416   2021-07-02  4357
947    2021-07-03  4105
479    2021-07-17  4638
10     2021-07-31  4696

[530 rows x 2 columns]


23:08:09 - cmdstanpy - INFO - Chain [1] done processing
23:08:09 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 6 Unknown error: -6
Optimization terminated abnormally. Falling back to Newton.
23:08:09 - cmdstanpy - INFO - Chain [1] start processing
23:08:09 - cmdstanpy - INFO - Chain [1] done processing
23:08:09 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 6 Unknown error: -6


RuntimeError: Error during optimization! Command '/Users/mohsinchougale/opt/anaconda3/lib/python3.9/site-packages/prophet/stan_model/prophet_model.bin random seed=9305 data file=/var/folders/ng/bpyglg5d53sczcphbzldc6rh0000gn/T/tmprmbsj35i/t53sdzk0.json init=/var/folders/ng/bpyglg5d53sczcphbzldc6rh0000gn/T/tmprmbsj35i/f0tzs8im.json output file=/var/folders/ng/bpyglg5d53sczcphbzldc6rh0000gn/T/tmprmbsj35i/prophet_modele8__r0td/prophet_model-20231007230809.csv method=optimize algorithm=newton iter=10000' failed: console log output:

dyld[66445]: Library not loaded: '@rpath/libtbb.dylib'
  Referenced from: '/Users/mohsinchougale/opt/anaconda3/lib/python3.9/site-packages/prophet/stan_model/prophet_model.bin'
  Reason: tried: '/private/var/folders/ng/bpyglg5d53sczcphbzldc6rh0000gn/T/pip-install-yah6sjxz/prophet_2d5405b0e9b94d40b0602fb2101e906f/build/lib.macosx-11.1-arm64-cpython-39/prophet/stan_model/cmdstan-2.31.0/stan/lib/stan_math/lib/tbb/libtbb.dylib' (no such file), '/private/var/folders